## Парсинг данных с сайта на Python

### Определения

Парсинг - это процесс сбора данных с последующей их обработкой и анализом.

Программа, которая занимается парсингом, называют - парсер.


### Условие задачи

С сайта ( https://habr.com/ru/search/ ) необходимо построить исходный набор данных (.csv или .xml). Набор данных должен включать __названия, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статей из Интернет-ресурсов__. Подготовленный набор данных должен содержать сведения о всех номинантах конкурса. Разработанный парсер должен извлекать гиперссылки из начальной страницы с последующим обходом всех страниц по полученным ссылкам и извлечением их содержимого. Можно дополнить набор какими-либо другими данными, если они могут быть полезны для дальнейшего исследования.


### Этапы парсинга

1. Поиск данных
2. Получение информации
3. Сохранение данных

### Подключение библиотек

In [387]:
from bs4 import BeautifulSoup as bs

Beautiful Soup - это библиотека Python для извлечения данных из HTML и XML файлов. 

In [389]:
import requests

Библиотека requests является стандартным инструментом для составления HTTP-запросов в Python.

In [391]:
import pandas as pd

### Получение информаций

In [393]:
# GET - запрос
url = 'https://habr.com/ru/all/' # страница со всеми статьями 
page = requests.get(url)

Метод __requests.get(url)__ из библиотеки requests в Python выполняет HTTP-запрос типа GET по указанному URL. Этот запрос используется для получения данных с веб-страницы или API, в нашем случае из страницы habr.

In [395]:
page.status_code

200

Если вызвать __page.status_code__, то получим статус состояния HTTP. например, 200 — успешно, 404 — страница не найдена, 500 — ошибка сервера 


In [397]:
soup = bs(page.text, 'html.parser')

__bs__ — это сокращение от BeautifulSoup, основного класса библиотеки Beautiful Soup.

__bs(page.text, 'html.parser')__ создаёт объект BeautifulSoup, который парсит HTML-код из page.text с использованием указанного парсера.

__'html.parser'__ — это встроенный парсер Python, который не требует установки дополнительных библиотек.а.

In [399]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n\n  <head>\n    <title>Все статьи подряд &#x2F; Хабр</title>\n<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">\n<link href="https://habr.com/ru/articles/" rel="canonical" data-hid="e3fa780">\n<link href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate" data-hid="7d51b8a">\n<link href="https://habr.com/en/articles/" hreflang="en" rel="alternate" data-hid="7d51b8a">\n<meta itemprop="image" content="/img/habr_ru.png">\n<meta property="og:image" content="/img/habr_ru.png">\n<meta property="og:image:width" content="1200">\n<meta property="og:image:height" content="630">\n<meta property="aiturec:image" content="/img/habr_ru.png">\n<meta name="twitter:image" content="/img/habr_ru.png">\n<meta property="vk:image" content="/img/habr_ru.png?format=vk">\n<meta property="fb:app_id" content="444736788986613">\n<meta property="fb:pages" content="472597926099084">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="tw

In [400]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Все статьи подряд / Хабр</title>
<link data-hid="2a79c45" href="/img/habr_ru.png" rel="image_src"/>
<link data-hid="e3fa780" href="https://habr.com/ru/articles/" rel="canonical"/>
<link data-hid="7d51b8a" href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate"/>
<link data-hid="7d51b8a" href="https://habr.com/en/articles/" hreflang="en" rel="alternate"/>
<meta content="/img/habr_ru.png" itemprop="image"/>
<meta content="/img/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="/img/habr_ru.png" property="aiturec:image"/>
<meta content="/img/habr_ru.png" name="twitter:image"/>
<meta content="/img/habr_ru.png?format=vk" property="vk:image"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@habr_com" name=

Создадим словарь, в который будем записывать данные по заданию: название статьи, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статьи из Интернет-ресурса

In [402]:
result_list = {'title': [], 'namecompany': [], 'description': [], 'rating': [], 'field': [], 'date': [], 'textpub': []}

### Алгоритм

Суть алгоритма заключается в переборе страниц, и переходе на "вложенные" страницы, то есть у нас есть основная страница https://habr.com/ru/all/, мы перебираем несколько стараниц с page1 до page10. На каждой странице есть статьи, записываем их в список, чтобы перейти по ним используем  _-i.a.get('href')-_  то есть берём значение из href этого заголовка. Далее находим классы элементов которые нам нужны, и записываем их в результат.

In [405]:
pagenum = 1
for i in range(3):
    url = 'https://habr.com/ru/articles/page' + str(pagenum) + '/' # переход на ссылуку с определённым номером сраницы
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h2', class_='tm-title tm-title_h2')# получаем заголовки всех статей на этой странице
    
    for i in titles: 
        # переход на страницу статьи
        url = 'https://habr.com' + str(i.a.get('href')) 
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        
        name_company = soup.find('a', class_='tm-company-snippet__title')# получаем название компаний
        desc_company = soup.find('div', class_='tm-company-snippet__description')# получаем описание компаний
        
        if (name_company is not None): #если на странице присутсвует компания
        
            result_list['title'].append(i.text) # записываем название статьи
            result_list['namecompany'].append(name_company.text) # записываем название компании
            result_list['description'].append(desc_company.text) # записываем описание компании
            
            datepub = soup.find('span', class_='tm-article-datetime-published') # находим дату публикаций
            result_list['date'].append(datepub.time['datetime'][0: 10]) # записываем дату публикаций
            
            # текст статьи
            try:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            except:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            result_list['textpub'].append(textpub)
            
            # переход на страницу компании
            url = 'https://habr.com' + str(name_company.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            
            #записываем рейтинг
            rating = soup.find('span', class_='tm-votes-lever__score-counter tm-votes-lever__score-counter_rating tm-votes-lever__score-counter')

            if(rating is None):
                result_list['rating'].append('0')
            else:
                result_list['rating'].append((rating.text).strip())
               
             #записываем отрасли компаний
            fieldtext = ""
            fields = soup.find_all('a', 'tm-company-profile__categories-text')
            for field in fields:
                fieldtext = fieldtext + ((field.text).strip()) + ", "
            if (fields is None):
                result_list['field'].append(None)
            else:
                result_list['field'].append(fieldtext[0:-2])
            
    pagenum += 1

In [406]:
result_list

{'title': ['Инхаус и аутсорс DevOps. Плюсы, минусы, подводные камни',
  '5 основополагающих советов по управлению базами данных Greenplum',
  'unittest.mock: 5 вопросов на собеседовании',
  'n8n. Создаём AI Telegram agent с установкой и настройкой',
  'Как Кремниевая долина превращает бум ИИ в новый пузырь доткомов',
  'Аннотации типов в Python: коротко о главном',
  'Бесплатная система учета торговли и управления интернет магазином Trinion Торговля',
  'Null-safety в Spring приложении с JSpecify и NullAway',
  'Использование JSONB-полей вместо EAV в PostgreSQL',
  'О книге «Разработка игр на Unity, 4-е изд.»',
  'Курсы по Роблокс для детей: цены и отзывы',
  'Школа программирования 8 класс: ТОП-10 школ',
  'Как мы строим дизайн-систему в ЮMoney',
  'Как OSA превращает пустые полки в полные корзины?',
  'Настройка BGP с Anycast: высокая доступность и отказоустойчивость',
  'Честные вакансии: путь от Junior DevOps к Middle и выше. Советы от экспертов Fournines, VK Tech и Kaspersky',
  '

In [407]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
title - 0
namecompany - 0
description - 0
rating - 0
field - 0
date - 0
textpub - 0


### Сохранение данных

In [409]:
file_name = 'habr.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [410]:
df.head()

,title,namecompany,description,rating,field,date,textpub
0,"Инхаус и аутсорс DevOps. Плюсы, минусы, подвод...",ITSumma,Эксперты в производительности,69.73,"Программное обеспечение, Консалтинг и поддержк...",2025-03-14,"Несмотря на то, что я почти десять лет работ..."
1,5 основополагающих советов по управлению базам...,OTUS,Цифровые навыки от ведущих экспертов,721.07,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2025-03-13,Greenplum — это массивно‑параллельная (MPP) ба...
2,unittest.mock: 5 вопросов на собеседовании,OTUS,Цифровые навыки от ведущих экспертов,721.07,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2025-03-13,"Привет, Хабр!Сегодня мы рассмотрим некоторые в..."
3,n8n. Создаём AI Telegram agent с установкой и ...,Amvera,Amvera — облако для хостинга IT-приложений,100.75,"Программное обеспечение, Домены и хостинг, Веб...",2025-03-13,Всем привет!В этой статье будет рассматриватьс...
4,Как Кремниевая долина превращает бум ИИ в новы...,RUVDS.com,VDS/VPS-хостинг. Скидка 15% по коду HABR15,2562.53,"Связь и телекоммуникации, Домены и хостинг, Ве...",2025-03-13,"Наблюдая за историей Кремниевой долины, всё оч..."


In [411]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        31 non-null     object
 1   namecompany  31 non-null     object
 2   description  31 non-null     object
 3   rating       31 non-null     object
 4   field        31 non-null     object
 5   date         31 non-null     object
 6   textpub      31 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB
